In [1]:
#Initial imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, plot_confusion_matrix
from sklearn.metrics import confusion_matrix, plot_roc_curve, precision_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier, StackingRegressor, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [24]:
df = pd.read_sas('../../Data/nsch_2020_topical.sas7bdat')

In [21]:
len(df)

42777

In [20]:
for col in df.columns:
    print(f'{col} null count: {df[col].isna().sum()}')
    print(f'{col} null percent: {df[col].isna().sum() / len(df)}\n')

FIPSST null count: 0
FIPSST null percent: 0.0

STRATUM null count: 0
STRATUM null percent: 0.0

HHID null count: 0
HHID null percent: 0.0

FORMTYPE null count: 0
FORMTYPE null percent: 0.0

TOTKIDS_R null count: 0
TOTKIDS_R null percent: 0.0

TENURE null count: 0
TENURE null percent: 0.0

HHLANGUAGE null count: 154
HHLANGUAGE null percent: 0.0036000654557355586

SC_AGE_YEARS null count: 0
SC_AGE_YEARS null percent: 0.0

SC_SEX null count: 0
SC_SEX null percent: 0.0

K2Q35A_1_YEARS null count: 41619
K2Q35A_1_YEARS null percent: 0.9729293779367417

BIRTH_MO null count: 266
BIRTH_MO null percent: 0.006218294878088692

BIRTH_YR null count: 385
BIRTH_YR null percent: 0.009000163639338896

MOMAGE null count: 926
MOMAGE null percent: 0.02164714683124109

K6Q41R_STILL null count: 32690
K6Q41R_STILL null percent: 0.7641957126493208

K6Q42R_NEVER null count: 30717
K6Q42R_NEVER null percent: 0.7180727961287607

K6Q43R_NEVER null count: 30762
K6Q43R_NEVER null percent: 0.7191247633073848

K6Q13A n

USUALSICK null count: 3233
USUALSICK null percent: 0.07557799752203287

K4Q31_R null count: 115
K4Q31_R null percent: 0.0026883605675947354

K4Q23 null count: 608
K4Q23 null percent: 0.014213245435631298

ALTHEALTH null count: 385
ALTHEALTH null percent: 0.009000163639338896

K4Q27 null count: 133
K4Q27 null percent: 0.003109147439044346

NOTELIG null count: 41315
NOTELIG null percent: 0.965822755218926

AVAILABLE null count: 41315
AVAILABLE null percent: 0.965822755218926

APPOINTMENT null count: 41315
APPOINTMENT null percent: 0.965822755218926

TRANSPORTCC null count: 41315
TRANSPORTCC null percent: 0.965822755218926

NOTOPEN null count: 41315
NOTOPEN null percent: 0.965822755218926

ISSUECOST null count: 41315
ISSUECOST null percent: 0.965822755218926

HOSPITALSTAY null count: 138
HOSPITALSTAY null percent: 0.0032260326811136826

K6Q15 null count: 188
K6Q15 null percent: 0.004394885101807046

SESCURRSVC null count: 36419
SESCURRSVC null percent: 0.8513687261846319

K4Q36 null count


A1_BORN null count: 1170
A1_BORN null percent: 0.0273511466442247

A1_EMPLOYED null count: 1453
A1_EMPLOYED null percent: 0.03396685134534914

A1_GRADE null count: 0
A1_GRADE null percent: 0.0

A1_MARITAL null count: 1364
A1_MARITAL null percent: 0.031886294036514946

A1_RELATION null count: 1086
A1_RELATION null percent: 0.02538747457745985

A2_SEX null count: 7998
A2_SEX null percent: 0.1869696332141104

A2_BORN null count: 8251
A2_BORN null percent: 0.1928840264628188

A2_EMPLOYED null count: 8137
A2_EMPLOYED null percent: 0.19021904294363792

A2_GRADE null count: 8081
A2_GRADE null percent: 0.18890992823246136

A2_MARITAL null count: 8126
A2_MARITAL null percent: 0.1899618954110854

A2_RELATION null count: 1301
A2_RELATION null percent: 0.03041353998644131

A1_ACTIVE null count: 1300
A1_ACTIVE null percent: 0.030390162938027446

A2_ACTIVE null count: 8057
A2_ACTIVE null percent: 0.18834887907052855

A1_PHYSHEALTH null count: 1342
A1_PHYSHEALTH null percent: 0.03137199897140987

A1

SC_SEX_IF null percent: 0.0

BIRTHWT_OZ_S null count: 1547
BIRTHWT_OZ_S null percent: 0.03616429389625266

BREASTFEDEND_DAY_S null count: 42655
BREASTFEDEND_DAY_S null percent: 0.9971480000935082

BREASTFEDEND_WK_S null count: 41326
BREASTFEDEND_WK_S null percent: 0.9660799027514786

BREASTFEDEND_MO_S null count: 33984
BREASTFEDEND_MO_S null percent: 0.7944456132968651

FRSTFORMULA_DAY_S null count: 38590
FRSTFORMULA_DAY_S null percent: 0.9021202982911377

FRSTFORMULA_WK_S null count: 36941
FRSTFORMULA_WK_S null percent: 0.8635715454566706

FRSTFORMULA_MO_S null count: 34023
FRSTFORMULA_MO_S null percent: 0.795357318185006

FRSTSOLIDS_DAY_S null count: 42443
FRSTSOLIDS_DAY_S null percent: 0.9921920658297684

FRSTSOLIDS_WK_S null count: 42296
FRSTSOLIDS_WK_S null percent: 0.9887556397129298

FRSTSOLIDS_MO_S null count: 31683
FRSTSOLIDS_MO_S null percent: 0.7406550248965565

HOUSE_GEN null count: 728
HOUSE_GEN null percent: 0.017018491245295368

FAMILY_R null count: 1111
FAMILY_R null pe

In [25]:
df.head()

,FIPSST,STRATUM,HHID,FORMTYPE,TOTKIDS_R,TENURE,HHLANGUAGE,SC_AGE_YEARS,SC_SEX,K2Q35A_1_YEARS,BIRTH_MO,BIRTH_YR,MOMAGE,K6Q41R_STILL,K6Q42R_NEVER,K6Q43R_NEVER,K6Q13A,K6Q13B,K6Q14A,K6Q14B,K4Q32X01,K4Q32X02,K4Q32X03,K4Q32X04,K4Q32X05,DENTALSERV1,DENTALSERV2,DENTALSERV3,DENTALSERV4,DENTALSERV5,DENTALSERV6,DENTALSERV7,K4Q28X01,K4Q28X02,K4Q28X03,K4Q28X_EAR,K4Q28X04,K4Q28X05,SESPLANYR,SESPLANMO,K4Q37,SPCSERVMO,LIVEUSA_YR,LIVEUSA_MO,K11Q43R,A1_AGE,A1_LIVEUSA,A2_AGE,A2_LIVEUSA,HHCOUNT,FAMCOUNT,BREATHING,SWALLOWING,STOMACH,PHYSICALPAIN,HANDS,COORDINATION,TOOTHACHES,GUMBLEED,CAVITIES,MEMORYCOND,WALKSTAIRS,DRESSING,ERRANDALONE,K2Q43B,BLINDNESS,ALLERGIES,ALLERGIES_CURR,ARTHRITIS,ARTHRITIS_CURR,K2Q40A,K2Q40B,K2Q61A,K2Q61B,K2Q41A,K2Q41B,K2Q42A,K2Q42B,HEART,HEART_BORN,HEART_CURR,HEADACHE,HEADACHE_CURR,K2Q38A,K2Q38B,K2Q33A,K2Q33B,K2Q32A,K2Q32B,DOWNSYN,BLOOD,SICKLECELL,THALASSEMIA,BLOOD_OTHER,BLOOD_SCREEN,CYSTFIB,CYSTFIB_SCREEN,GENETIC,GENETIC_SCREEN,K2Q34A,K2Q34B,K2Q36A,K2Q36B,K2Q60A,K2Q60B,K2Q37A,K2Q37B,K2Q30A,K2Q30B,K2Q35A,K2Q35B,AUTISMMED,AUTISMTREAT,K2Q31A,K2Q31B,K2Q31D,ADDTREAT,CONCUSSION,SEEKCARE,CONFIRMINJURY,K2Q05,K6Q40,S4Q01,DOCPRIVATE,OVERWEIGHT,K6Q10,K6Q12,K4Q01,USUALGO,USUALSICK,K4Q31_R,K4Q23,ALTHEALTH,K4Q27,NOTELIG,AVAILABLE,APPOINTMENT,TRANSPORTCC,NOTOPEN,ISSUECOST,HOSPITALSTAY,K6Q15,SESCURRSVC,K4Q36,K4Q38,K5Q10,DECISIONS,K5Q21,TREATCHILD,TREATADULT,MEDHISTORY,WRITEPLAN,RECEIVECOPY,HEALTHKNOW,KEEPINSADULT,K12Q01_A,K12Q01_B,K12Q01_C,K12Q01_D,K12Q01_E,K12Q01_F,K12Q01_G,CURRCOV,K12Q03,K12Q04,K12Q12,TRICARE,K11Q03R,HCCOVOTH,K3Q25,STOPWORK,CUTHOURS,AVOIDCHG,ONEWORD,TWOWORDS,THREEWORDS,ASKQUESTION,ASKQUESTION2,TELLSTORY,UNDERSTAND,DIRECTIONS,POINT,DIRECTIONS2,UNDERSTAND2,RHYMEWORD,REPEATED,K7Q30,K7Q31,K7Q32,K7Q37,K7Q38,BORNUSA,K8Q35,EMOSUPSPO,EMOSUPFAM,EMOSUPHCP,EMOSUPWOR,EMOSUPADV,EMOSUPPEER,EMOSUPMHP,EMOSUPOTH,K6Q20,K6Q27,K9Q40,K9Q41,K11Q60,K11Q61,K11Q62,S9Q34,K10Q11,K10Q12,K10Q13,K10Q14,K10Q20,K10Q22,K10Q23,K9Q96,ACE3,ACE4,ACE5,ACE6,ACE7,ACE8,ACE9,ACE10,ACE12,A1_DEPLSTAT,A2_DEPLSTAT,ALLERGIES_DESC,ARTHRITIS_DESC,K2Q40C,CERPALS_DESC,K2Q41C,K2Q42C,HEART_DESC,HEADACHE_DESC,K2Q38C,K2Q33C,K2Q32C,BLOOD_DESC,GENETIC_DESC,K2Q34C,K2Q36C,K2Q60C,K2Q37C,K2Q30C,K2Q35C,K2Q31C,A1_SEX,A1_BORN,A1_EMPLOYED,A1_GRADE,A1_MARITAL,A1_RELATION,A2_SEX,A2_BORN,A2_EMPLOYED,A2_GRADE,A2_MARITAL,A2_RELATION,A1_ACTIVE,A2_ACTIVE,A1_PHYSHEALTH,A1_MENTHEALTH,A2_PHYSHEALTH,A2_MENTHEALTH,K2Q01,K2Q01_D,K5Q40,K5Q41,K5Q42,K5Q43,K5Q44,DISCUSSOPT,RAISECONC,BESTFORCHILD,K3Q20,K3Q22,K3Q21B,K6Q70_R,K6Q73_R,K6Q71_R,K6Q72_R,K7Q84_R,K7Q85_R,K7Q82_R,K7Q83_R,K7Q70_R,BULLIED_R,BULLY,RECOGBEGIN,CLEAREXP,WRITENAME,RECSHAPES,DISTRACTED,WORKTOFIN,SIMPLEINST,PLAYWELL,NEWACTIVITY,HURTSAD,CALMDOWN,TEMPER,SITSTILL,RECOGABC,TALKABOUT,WKTOSOLVE,STRENGTHS,HOPEFUL,K10Q30,K10Q31,K10Q40_R,GOFORHELP,K10Q41_R,K8Q31,K8Q32,K8Q34,HOWMUCH,ATHOMEHC,ARRANGEHC,K7Q02R_R,K7Q04R_R,PHYSACTIV,HOURSLEEP05,HOURSLEEP,SCREENTIME,K6Q60_R,K6Q61_R,K8Q11,FOODSIT,POSCHOICE,GAINSKILLS,CHANGEAGE,K2Q35D,HCABILITY,HCEXTENT,K4Q20R,DOCROOM,WGTCONC,K4Q02_R,DENTISTVISIT,K4Q22_R,TREATNEED,K4Q24_R,K4Q26,C4Q04,HOSPITALER,K4Q04_R,K5Q11,K5Q20_R,K5Q22,K5Q30,K5Q32,K5Q31_R,K8Q21,K8Q30,COUNTTO,K7Q33,BEDTIME,K3Q04_R,K6Q08_R,CONFIDENT,ACE1,USEPENCIL,MAKEFRIEND,SLEEPPOS,COLOR,K4Q30_R,STARTSCHOOL,MENBEVCOV,PLANNEEDS_R,YEAR,CBSAFP_YN,METRO_YN,MPC_YN,TOTAGE_0_5,TOTAGE_6_11,TOTAGE_12_17,TOTCSHCN,TOTNONSHCN,SC_RACE_R,SC_HISPANIC_R,SC_CSHCN,SC_ENGLISH,SC_K2Q10,SC_K2Q11,SC_K2Q12,SC_K2Q13,SC_K2Q14,SC_K2Q15,SC_K2Q16,SC_K2Q17,SC_K2Q18,SC_K2Q19,SC_K2Q20,SC_K2Q21,SC_K2Q22,SC_K2Q23,SC_AGE_LT4,SC_AGE_LT6,SC_AGE_LT9,SC_AGE_LT10,AGEPOS4,TENURE_IF,TOTMALE,TOTFEMALE,SC_RACE_R_IF,SC_RACER,SC_RACEASIA,SC_RACEAIAN,SC_HISPANIC_R_IF,SC_SEX_IF,BIRTHWT_OZ_S,BREASTFEDEND_DAY_S,BREASTFEDEND_WK_S,BREASTFEDEND_MO_S,FRSTFORMULA_DAY_S,FRSTFORMULA_WK_S,FRSTFORMULA_MO_S,FRSTSOLIDS_DAY_S,FRSTSOLIDS_WK_S,FRSTSOLIDS_MO_S,HOUSE_GEN,FAMILY_R,CURRINS,INSGAP,INSTYPE,HIGRADE,HIGRADE_TVIS,BIRTHWT_VL,BIRTHWT_L,BIRTHWT,FPL_IF,A1_GRADE_IF,BIRTH_YR_F,BMICLASS,HHCOUNT_IF,FPL_I1,FPL_I2